In [ ]:
import mib_v2_3_thr as mb2_3tr
from itertools import product
import time
wn = 50
 
n_vocab = 58

time_Q_TAW_nv = []

for i in [10,11,12,13]:
    time_Q_TAW = []

    A = mb2_3tr.Var('A', set([j for j in range(10)]))
    T = mb2_3tr.Var('T', set([0,1,2]))
    W = [mb2_3tr.Var(j, set([0,1])) for j in range(i)]
    
    print("n_vocab: ",i)
    dA = {}
    for j in range(10):
        dA[(j,)] = 1/10
        
    PA = mb2_3tr.Distrib(set([A]), dA, ('A'))
    
    
    dT_A = {}
    for j in range(10):
        dT_A[(j,)] = {}
        for k in range(3):
            dT_A[(j,)][(k,)] = 1/10
        
    PT_A = mb2_3tr.CondDistrib(set([T]), set([A]),dT_A, ('T',), ('A'))
    
    PW_AT = []
    for wi in W:
        dwi_TA = {}
        for par in product(*[[0,1,2], [j for j in range(10)]]):
            dwi_TA[par] = {}
            for j in range(2):
                dwi_TA[par][(j,)] = 0.5

        PWi_AT = mb2_3tr.CondDistrib(set([wi]), set([T,A]), dwi_TA, (wi.getName(),), ('T','A'))
        PW_AT.append(PWi_AT)
    
    set_vars = set([A,T] + W)
    set_descomp = set([PA, PT_A,] + PW_AT)
    PATW = mb2_3tr.Specification(set_vars,set_descomp)
    
    QW_AT = mb2_3tr.Question(PATW)
    
    for j in range(i):
        Wt = [W[k-1] for k in range(j+1)]
        Wt_v = [1 for k in range(j+1)]
        
        inicio = time.time()
        CA, VA, p = QW_AT.Query(vars=(A,), indep=tuple(Wt), values_indep=tuple(Wt_v))
        fin = time.time()
        print("n_w:",j,"-",fin - inicio)
        time_Q_TAW.append(fin - inicio)
    
    time_Q_TAW_nv.append(time_Q_TAW)